In [1]:
import numpy as np
import matplotlib.pyplot as plt
import jax
import copy
import queue
from shower_sim_instrumented import simulator
from shower_sim_baseline import summary_metric

%load_ext autoreload
%autoreload 2

In [2]:
hits,active,history,scores,out_st = simulator(3.5)
print(scores, out_st['w'], hits.size, out_st['y']['hits'].size)

0.87279123 48.394210412632674 2736 2848


In [3]:
par_vals = np.arange(0.25, 4.0, 0.1)
N=200
eps = 0.01

primal_list = []
primal_st_list = []

score_list = []
stad_list = []
numeric_list = []
dlogp_list = []


for par_v in par_vals:
    print("############ par value=", par_v, "##################")
    
    primal = []
    primal_st = []

    score_val = []
    stad_val = []
    numeric_val = []
    dlogp_val = []
    
    for i in range(N):
        if i%100 == 0: print("############", i, "##################")
        hits,active,history,scores,out_st = simulator(par_v)
        _, active2, _, _, _ = simulator(par_v+eps)
        
        _val = summary_metric(active)
        _val2 = summary_metric(active2)
        
        primal.append(_val)
        primal_st.append(summary_metric(out_st['y']['active']))
        
        numeric_val.append( (_val2 - _val)/eps )
        
        score_val.append(scores*primal[i])
        dlogp_val.append(scores)
        stad_val.append(out_st['d'] + out_st['w']*(primal_st[i] - primal[i]))
        
    primal_list.append(primal)
    primal_st_list.append(primal_st)
    
    score_list.append(score_val)
    stad_list.append(stad_val)
    numeric_list.append(numeric_val)
    dlogp_list.append(dlogp_val)
    

primal_list = np.array(primal_list)
primal_list_m = primal_list.mean(axis=1)
primal_list_s = primal_list.std(axis=1)

numeric_list = np.array(numeric_list)
numeric_m = numeric_list.mean(axis=1)
numeric_s = numeric_list.std(axis=1)

score_list = np.array(score_list)
score_m = score_list.mean(axis=1)
score_s = score_list.std(axis=1)

dlogp_list = np.array(dlogp_list)
score_baseline_list = score_list - dlogp_list*primal_list_m.reshape(-1,1)
score_baseline_m = score_baseline_list.mean(axis=1)
score_baseline_s = score_baseline_list.std(axis=1)

stad_list = np.array(stad_list)
stad_m = stad_list.mean(axis=1)
stad_s = stad_list.std(axis=1)

############ par value= 0.25 ##################
############ 0 ##################
############ 100 ##################
